In [2]:
library(here, help, pos = 2, lib.loc = NULL)

here() starts at C:/Users/arthu/source/repos/thj3a/Barreiras-e-Facilitadores/Barreiras-e-Facilitadores



In [3]:
source(here('Modules','modules.R'))

In [ ]:
install_all_packages()

In [4]:
load_library_packages()

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: 'randomForest'


The following object is masked from 'package:ggplot2':

    margin


Carregando pacotes exigidos: lattice

Carregando pacotes exigidos: rpart

Loaded ROSE 0.0-4



Attaching package: 'DescTools'


The following objects are masked from 'package:caret':

    MAE, RMSE



Attaching package: 'dplyr'


The following object is masked from 'package:MASS':

    select


The following object is masked from 'package:xgboost':

    slice


The following object is masked from 'package:randomForest':

    combine


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'kernlab'


The following object is masked from 'package:ggplot2':

    alpha


dummies-1.5.6 provided by Decision Patterns



--------------------------------------------------------------

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[9]]
[1] TRUE

[[10]]
[1] TRUE

[[11]]
[1] TRUE

[[12]]
[1] TRUE

[[13]]
[1] TRUE

[[14]]
[1] TRUE

[[15]]
[1] TRUE

[[16]]
[1] TRUE

[[17]]
[1] TRUE

[[18]]
[1] TRUE

[[19]]
[1] TRUE

[[20]]
[1] TRUE

[[21]]
[1] TRUE

[[22]]
[1] TRUE

[[23]]
[1] TRUE

[[24]]
[1] TRUE

In [5]:
BD = read_excel(path = here('Data','Physical Activity_BEFORE.xlsx'), sheet = "Planilha1")

In [6]:
create_model(BD=BD,TipoModelo='rpart',min_classes=c("Doesnt.Practiced.PA"),maj_class=c("Practiced.PA"), model_metric='Kappa', trn_ctrl_method='boot')

CART 

867 samples
 19 predictor
  2 classes: 'Doesnt.Practiced.PA', 'Practiced.PA' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 867, 867, 867, 867, 867, 867, ... 
Resampling results across tuning parameters:

  cp           Accuracy   Kappa     
  0.000000000  0.8424300  0.06950238
  0.004160888  0.8485314  0.06617989
  0.007281553  0.8539841  0.04377411

Kappa was used to select the optimal model using the largest value.
The final value used for the model was cp = 0.
CART 

1527 samples
  19 predictor
   2 classes: 'Doesnt.Practiced.PA', 'Practiced.PA' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 1527, 1527, 1527, 1527, 1527, 1527, ... 
Resampling results across tuning parameters:

  cp          Accuracy   Kappa     
  0.04521625  0.8053186  0.61123403
  0.07404980  0.7267940  0.45793090
  0.39580603  0.5253830  0.07085346

Kappa was used to select the optimal model using the largest value.
The final value used f

[1] "C:/Users/arthu/source/repos/thj3a/Barreiras-e-Facilitadores/Barreiras-e-Facilitadores/Modules/modules.R"